<a href="https://colab.research.google.com/github/RakibAkram5/AgenticAI/blob/main/Assignment_NO_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet google-genai

In [2]:
!pip install gTTs --quiet

In [3]:
!pip install playsound --quiet

  Preparing metadata (setup.py) ... done


In [15]:
from google.colab import userdata
GOOGLE_API_KEY: str=userdata.get('GOOGLE_API_KEY')
if(GOOGLE_API_KEY):
  print("API KEY FOUND")
else:
  print("API KEY NOT FOUND")

API KEY FOUND


In [16]:
from google.genai.types import GenerateContentResponse
from IPython.display import display, Markdown

In [18]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key = GOOGLE_API_KEY,
)

model: str = "gemini-2.0-flash-exp"

In [23]:
video_file_name ="/content/WhatsApp Video 2024-12-28 at 11.12.53_1838b2e1.mp4"

In [24]:
import time


def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

my_video = upload_video(video_file_name)

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/55ih9jbu0vml


In [25]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions that describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = my_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)

Markdown(response.text)

```json
[
  {
    "timecode": "00:00",
    "caption": "A man wearing a black jacket with a white, orange, and green pattern stands indoors, speaking to the camera. He has dark hair and a beard. \"Assalamualikum myself, Faki Bakra, I'm from Lahore. I'm doing my Bachelors in Computer Science from UCP. I have three siblings. I'm the elder one.\""
  },
  {
    "timecode": "00:10",
    "caption": "The man continues talking while smiling, with the same indoor background. \"And, I have three friends, or sorry, four friends in my...\""
  }
]
```

In [26]:
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio


tts = gTTS(text=response.text, lang='en')

with open('output.mp3', 'wb') as f:
  for chunk in tts.stream():
    f.write(chunk)

display(Audio('output.mp3', autoplay=True))